## 1 简介

接着上一个项目 ，[用文心大模型飞飞机3-多模态感知大模型](https://aistudio.baidu.com/projectdetail/7370323)

前面3个项目，我们是在airsim仿真系统里面，测试文心大模型驱动airsim无人机飞行，这一个项目则使用真实的tello无人机，选择前面项目的几个典型例子进行测试。

把虚拟世界的无人机大模型迁移到真实无人机上，相对比较简单，只需要把对应的控制函数和感知函数，进行对应修改即可，其他的大模型感知、推理则无需任何改变。

我们使用的tello微型无人机，只有80克，室内飞不用备案，而且非常稳定。当然，主要还是支持python编程控制。

具体有下面3个型号，白色的最便宜，新的699，二手的某鱼二三百即可，其他两个功能差不多，但价格为999、1999，感觉不是很划算。

当然还有更便宜的开源Crazyflie/minifly无人机，三四百就能买个新的，也是支持各类代码编程的:


<img src="https://ai-studio-static-online.cdn.bcebos.com/3c69b343d8b646df820a2b3acd14a560dae9e1f2aeb74ec881817daa4e6ba32e" width="600" height="600" />


</br>





我们主要测试两个典型任务：

1. 自行感知，找到玩具小鸭子，并飞过去。
2. 到楼上看看，窗户关了没。


重点问题：
1. 控制和视觉 sdk的替换，从airsim虚拟无人机替换为tello真实无人机的sdk。prompt大部分无需替换。
2. tello为单目视觉，没有深度相机或激光雷达，因此，深度距离需要估算，这一点是和airsim中无人机是不同的。



本项目的场景用的是我们一个办公点的场景,一个楼下一个楼上：

<img src="https://ai-studio-static-online.cdn.bcebos.com/0a46abb1b9884edc9a0173bfd951111821a6a81180034e0f9845abde2efb993a" width="600" height="600" />


</br>



<img src="https://ai-studio-static-online.cdn.bcebos.com/a6cb1a7f73384f5988c5708542e42c30a5d07291dc834a1da614f5ed73e9512f" width="600" height="600" />


</br>



## 2 tello无人机的控制/感知sdk

具体的文心大模型的prompt工程封装，见左侧文件ernie_airsim.py，需要把其中对应的文心token改成自己的即可。

我们使用 [DJITelloPy](https://github.com/damiafuentes/DJITelloPy) 作为tello无人机的控制sdk，然后进行prompt描述即可。

作为无人机的基本控制行为，我们只需要下面几个通用的即可：

行为：

1. 起飞：tello.takeoff()
2. 降落：tello.land()
3. 向前：tello.move_forward(distance)
4. 向后：tello.move_back(distance)
5. 向左转：tello.rotate_counter_clockwise(degree)
6. 向右转：tello.rotate_clockwise(degree)
7. 向上：tello.move_up(distance)
8. 向下：tello.move_down(distance)

感知：
1. 获得摄像头数据。tello.get_frame_read()
2. 获得无人机位姿。tello.query_attitude()

然后我们对这些动作和感知函数进行统一的封装，也就是和airsim中的无人机封装函数名字保持一致即可，这样，文心无人机大模型的

ernie_airsim.py调用基本不需要修改，即可直接操作tello无人机了。具体的无人机sdk封装见tello_wrapper_ob.py。


## 3 单目视觉定位

理论上讲，只有双目摄像头，才能精准的进行距离判定，也就是计算目标距离摄像头的距离。tello头部只有一个摄像头，也就是单目的，而单目是没法精准判断的，但有了深度学习，只要有大量的标注数据，也可以预估啊。

单目深度估计是一项计算机视觉任务，涉及从单个图像预测场景的深度信息。换句话说，它是从单个摄像机视点估计场景中物体距离的过程。

单目深度估计有多种应用，包括 3D 重建、增强现实、自动驾驶和机器人技术。这是一项具有挑战性的任务，因为它要求模型理解场景中物体之间的复杂关系以及相应的深度信息，

这些关系可能受到光照条件、遮挡和纹理等因素的影响。目前常用的估算模型有DPT, GLPN等，我们以GLPN为例，测试下距离估算的结果。
、

In [ ]:
#aistudo 的notebook无法直接执行，需要改成python文件执行
from transformers import pipeline
checkpoint = "vinvino02/glpn-nyu"
depth_estimator = pipeline("depth-estimation", model=checkpoint)

In [ ]:
from PIL import Image
import requests
 
img = "color_image.jpg"
image = Image.open(img)
image

<img src="https://ai-studio-static-online.cdn.bcebos.com/80a5a92fb6b5401e9c5a0d0e01cc8312ee087132c9ae446a93870b90d6459be5" width="600" height="600" />



In [ ]:
predictions = depth_estimator(image)
predictions["depth"] #深度图

<img src="https://ai-studio-static-online.cdn.bcebos.com/62dd48ee6e33459691ff19f240dbf5f248ee8cdd69544847b58d28c3107dbef5" width="600" height="600" />



In [ ]:
predictions["predicted_depth"] #每个像素点距离摄像头的距离

"""
tensor([[[4.5112, 4.2299, 4.2204,  ..., 4.0969, 4.1953, 4.4710],
         [3.8822, 3.3689, 3.1847,  ..., 3.1596, 3.3350, 3.7277],
         [4.1643, 3.9953, 3.9554,  ..., 3.6966, 3.8663, 3.8989],
         ...,
         [1.2474, 1.2855, 1.2984,  ..., 1.3921, 1.4053, 1.3963],
         [1.2805, 1.2214, 1.3137,  ..., 1.3758, 1.4909, 1.3620],
         [1.3612, 1.3406, 1.2530,  ..., 1.4052, 1.4514, 1.4885]]])
"""

## 4 任务1：寻找小黄鸭

和前面airsim虚拟环境中，寻找小黄鸭的任务类似，只是我们改用了真实的无人机。

注意执行的时候，把大模型处理中process函数的python代码执行模块打开。

In [6]:
#初始化无人机大模型
import ernie_airsim
my_ernie_airsim = ernie_airsim.ErnieAirSim(system_prompts='system_prompts/airsim_objectnavigation_cn.txt', prompt='prompts/airsim_objectnavigation_cn.txt')

#初始化无人机sdk
from tello_wrapper_ob import *
tello = TelloWrapper() #tello基本功能调用

In [ ]:
#无人机依次执行如下指令，直接执行llm生成的代码，然后观测无人机的飞行情况
command_list = [
"起飞",
"我需要你帮我找到一个对象。对象可能散布在场景中，因此，如果您没有立即找到它，一个好的策略是转身几次，看看它是否变得可见。给我代码来探索并找到小鸭子",
"太棒了！现在假设我们确实找到了小鸭子。现在我想向它迈进。我们拥有的唯一信息是图像中瓶子边界框的位置。你能给我代码来实现这一点吗？",
"降落"
]

for command in command_list:
    response = my_ernie_airsim.process(command)
    print(response)


tello无人机执行的情况大致如下：

主要的决策感知图像：


<img src="https://ai-studio-static-online.cdn.bcebos.com/3d298e74ed2b4463a670393c38b1f557d685ea34164947528d5423e3a5cc7528" width="600" height="600" />


</br>


具体的执行视频如下：


<video height="400" width="600" poster="https://ai-studio-static-online.cdn.bcebos.com/3d298e74ed2b4463a670393c38b1f557d685ea34164947528d5423e3a5cc7528" preload="none" controls="" id="video">
      <source type="video/mp4" src="https://airgpt.bj.bcebos.com/tello-find-toy.mp4" id="mp4">
</video>



可以看到，在文心大模型的驱动下，真实的无人机也同样能顺利完成任务，当然，我们在实验中，也多次出现无人机飞行偏离预定航线，或者都快碰到桌子了，

pc端无法及时调整反馈的情况，这主要也是因为真实的物理环境，还是要比仿真环境更为复杂，特别是室内物品较多的环境下。





## 5 任务2：室内巡检

室内巡检和上一个实验“到楼上看看，窗户关了没”属于同一个任务，只是从airsim仿真环境， 转移到了真实的环境中。

同样也是需要无人机自主处理两类任务。

1 看看环境中都有什么物品。

2 根据物品位置和目标，确定飞行路径。

整个流程全部由无人机自主完成，基本流程就是：

1. 确定任务。
2. 环境感知，获得物品及其位置列表，如[('yellow duck', 15.375, -21.425346762452097), ('flower', 14.7421875, -42.059604476553346)......]
3. 文心大模型根据感知信息，为无人机确定具体的行为action。
4. 重复2和3，直到完成任务。



具体代码如下：


In [ ]:
from tello_wrapper_ob import *
tello = TelloWrapper() #tello基本功能调用

#初始化无人机大模型
import ernie_airsim
my_ernie_airsim = ernie_airsim.ErnieAirSim(system_prompts='system_prompts/airsim_ob.txt', prompt='prompts/airsim_ob1.txt')


command = """
任务：到楼上办公室看看，窗户关了没有
"""
response = my_ernie_airsim.process(command)


while True:
    result = tello.ob_objects_llm(obj_name_list) #获得物品列表及其位置
    command = """
            场景中的物体：{}
            命令：
            """.format(result)
    response = my_ernie_airsim.process(command)

    if "windows" in result: #如果发现窗户，就退出，表示完成了任务。可以用prompt构造判断，进行改进。
        break 

tello无人机执行的情况大致如下：

主要的决策感知图像如下：


<img src="https://ai-studio-static-online.cdn.bcebos.com/5006706aee21493faae780bf568100b734b6aa55fc084ea98e08016e1590d51d" width="1000" height="600" />


</br>

图片总共有两行，第一行为具体的多模态感知图像，第二行为估算的深度距离图。



具体的执行视频如下：


<video height="400" width="600" poster="https://ai-studio-static-online.cdn.bcebos.com/3d298e74ed2b4463a670393c38b1f557d685ea34164947528d5423e3a5cc7528" preload="none" controls="" id="video">
      <source type="video/mp4" src="https://airgpt.bj.bcebos.com/tello-get-stair.mp4" id="mp4">
</video>



在复杂场景下，tello无人机先进行多模态感知，然后再依据文心大模型进行行为决策，因为感知的输出有一定的不确定性，而单目相机进行距离预测也不是很准，

因此无人机实际在上楼梯的时候，经常会碰到楼梯，或者左右移动停止不前，这也说明了大模型进行决策时的不确定性。

上面遇到的问题，是无人机应用大模型时最明显的2个问题。解决这些问题，仍需要大模型推理能力的进一步增强，以及多模态感知的进一步优化。

## 6 小结

这个项目把 “文心大模型+(RAM+GroundingDINO)多模态感知模型” 构建的典型Agent框架，应用到tello真实无人机上，

虽然实际效果差强人意，但也明确验证了，大模型的无人应用已经度过了0到1的阶段，正在1到100的道路上狂飙。

本项目的推理，目前是在带4090显卡的pc机上运行的，pc机通过wifi和tello无人机进行交互，而更具实用价值的应用方式，

是直接在机载计算机上运行大模型，目前如nvidia jetson orin系列，高阶版本已经可以运行transformer类大模型，

相信随着边缘计算算力的进一步增强，越来越多的大模型会在无人机上得到应用。